# 1. Simple "LUBO" Example
### James Saslow
### 12/3/2023

The primary motivation of quantum annealers is to solve Quadratic Unconstrained Binary Optimization problems (QUBO). 

The goal to solving QUBO problems is to find the optimal solution to the following cost function:


### $C(X) = \sum_{i} W_{i} x_{i} + \sum_{\{i,j\}\epsilon \mathbb{S}} w_{i,j} x_{i} x_{j}$ 

where $X = \{x_{0}, x_{1}, ... x_{N-1} \}$ and $x_{i} \epsilon \{0,1\}$


In this example, we will omit the quadratic terms for simplicity such that we only have *linear* terms. Thus, we're left with a Linear Unconstrained Binary Optimization problem (Don't worry, we'll add back the quadraticness later!). A LUBO can essentially be thought of as a degenerate QUBO.

### The goal of the LUBO is to find the optimal solutions of the following cost function:

### $C(X) = \sum_{i}^{N} W_{i} x_{i}$

Let's analyze a specific example. Here's a LUBO I just came up with on the spot

### $C(|x_{0} x_{1} x_{2}>) = x_{0} - 2 x_{1} + 5 x_{2}$

### A question of interest is what binary bit string $|x_{0} x_{1} x_{2}>$ will minimize $C(X)$?

Well, upon inspection, if we set $x_{0} = 0$, $x_{1} = 1$, $x_{2} = 0$, we get $C(|010>) = 0 -2*1 +5*0 = -2$, which is the minimum of $C(X)$!

Let's Test this on DWave's Quantum Annealer Hybrid Solver and confirm that $|010>$ really is the solution to this LUBO.

In [1]:
# Importing DWave Packages
from dwave.system import DWaveSampler, EmbeddingComposite
from dimod import BinaryQuadraticModel

In [1]:
# Defining the QUBO parameters

items   = [0,1,2]  # Binary Variable Labeling
weights = [1,-2,5] # W weights associated with each binary variable

x = ['Item_0', 'Item_1', 'Item_2'] # More Binary Variable Labeling

In [3]:
# Encoding the QUBO in DWave solvers

bqm = BinaryQuadraticModel("BINARY") # Encoding QUBO

# Writing the Objective Function (adding each variable and assigning it's associated weight)
for i in items:
    bqm.add_variable(x[i], weights[i])
    
    
bqm

BinaryQuadraticModel({'Item_0': 1.0, 'Item_1': -2.0, 'Item_2': 5.0}, {}, 0.0, 'BINARY')

In [4]:
# Solve the problem using D-Wave
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=50)

In [5]:
# Post-Processing Solver Output
opt_solution = sampleset.first.sample # Binary Bit-String that minimizes the cost function
opt_value = sampleset.first.energy    # Minimized Cost function value

In [6]:
print(opt_solution)
print('C_min = ', opt_value)

{'Item_0': 0, 'Item_1': 1, 'Item_2': 0}
C_min =  -2.0


Here, we confirmed with DWave that the binary bit string that minimizes the given cost function is $|010>$ and $C_{min} = -2$

### But what if we want to find which binary bit string $|x_{0} x_{1} x_{2}>$ will *maximize* C(X)?

Quantum annealers will always construct an ising model equivalent of the QUBO that will relax to the ground state finding the minimum. 

So, we need to define a new cost function $C'(X)$ whose minimum is the maximum of $C(X)$.

To do this, we just need to carry out the following transformation on the $C(X)$ cost function: $C'(X)  = - C(X)$

Thus, the new cost function is

### $C'(X) = -x_{0} + 2 x_{1} - 5 x_{2}$

and we want to find the binary bit string $|x_{0}x_{1}x_{2}>$ that minimizes $C'(X)$, which is the same procedure as last time, just with a different *weights* vector.

In [7]:
# Defining the QUBO parameters

items   = [0,1,2]  # Binary Variable Labeling

# Weights for this C'(X) cost function are the negative of the C(X) cost function
weights = [-1,2,-5] # W weights associated with each binary variable

x = ['Item_0', 'Item_1', 'Item_2'] # More Binary Variable Labeling

In [8]:
# Encoding the QUBO in DWave solvers

bqm = BinaryQuadraticModel("BINARY") # Encoding QUBO

# Writing the Objective Function (adding each variable and assigning it's associated weight)
for i in items:
    bqm.add_variable(x[i], weights[i])
    
    
bqm

BinaryQuadraticModel({'Item_0': -1.0, 'Item_1': 2.0, 'Item_2': -5.0}, {}, 0.0, 'BINARY')

In [11]:
# Solve the problem using D-Wave
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm, num_reads=50)

In [12]:
# Post-Processing Solver Output
opt_solution = sampleset.first.sample # Binary Bit-String that minimizes the cost function

In [13]:
print(opt_solution)

{'Item_0': 1, 'Item_1': 0, 'Item_2': 1}


### Thus the solution that minimizes C'(X) is |101>

### This means that the binary bit string that maximizes C(X) is |101>

Let's evaluate $C(X)$ at $X = |101>$.

$C(X) = x_{0} - 2 x_{1} + 5 x_{2}$

$C(|101>) = (1) - 2 (0) + 5(1) = 6$


_________________________________

### Therefore, the optimal solutions to the LUBO $C(X) = x_{0} - 2 x_{1} +5 x_{2}$ are
- $X_{min} =  |010>$ , $C_{min} = C(|010>) = -2$
- $X_{max} =  |101>$ , $C_{max} = C(|101>) = 6$

Stay tuned for the next example where we impliment the Subset Sum Problem -- a real problem that can be formulated as a LUBO!

### Sources

[1] “Binary quadratic models#,” Binary Quadratic Models - Ocean Documentation 6.7.1 documentation, https://docs.ocean.dwavesys.com/en/stable/concepts/bqm.html (accessed Dec. 11, 2023). 

[2] “Quantum programming tutorial | D-wave qubits 2021,” YouTube, https://www.youtube.com/watch?v=jTDnGox0c9Y (accessed Dec. 11, 2023). 